In [1]:
import sys
sys.path.append('/home/li.yu/code/JupiterCVML/europa/base/src/europa')

In [2]:
import os
os.environ["BRT_ENV"] = 'prod'
import brtdevkit
brtdevkit.log = 'info'

from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import *
from aletheia_dataset_creator.config.dataset_config import *
from aletheia_dataset_creator.visualization_tools.aletheia_visualization_helpers import *
from dl.dataset.brt_dataset_helpers import *
from brtdevkit.core.db import DBConnector
from brtdevkit.core.db.db_filters import *
from brtdevkit.data import AnnotationJob, LabelMap
from brtdevkit.data import Dataset
from brtdevkit.core import DBConnector as dbc


from datetime import datetime
from collections import Counter
from bson import ObjectId
import pandas as pd
import numpy as np

# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Internal AWS credentials will be removed from future brt-devkit versions.


In [4]:
# !pip install pygeohash
#!pip install seaborn
import random
import pygeohash as pgh
# import seaborn as sns

#sudo apt-get install python3.8-dev
#!pip install polygon-geohasher==0.0.1



# from shapely import wkt
#! pip install geopandas
# from polygon_geohasher import polygon_geohasher
import geopandas as gpd
import folium
# from polygon_geohasher.polygon_geohasher import geohash_to_polygon


# ! pip install geolib
from geolib import geohash
# import geopandas as gpd
from shapely.geometry import Polygon


ModuleNotFoundError: No module named 'geopandas'

In [4]:
import json
def string_to_dict(dict_string):
    # Convert to proper json format
    dict_string = dict_string.replace("'", '"').replace('u"', '"')
    return json.loads(dict_string)

# train42p.CoordDicts = train42p.gnss.apply(string_to_dict)

def get_geohash(image_df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns dataframe with extracted latitude and longitude columns.
    :param image_df: Dataframe with image metadata
    """
    for index, img in image_df.iterrows():
        if img['gnss']:
            point_coordinates = string_to_dict(img.gnss)
            lon, lat = point_coordinates['coordinates'][0], point_coordinates['coordinates'][1]
#             lon, lat = img['gnss'].get('coordinates')[0], img['gnss'].get('coordinates')[1]
            image_df.at[index, 'geohash'] = pgh.encode(lat, lon)
        else:
            lon, lat = img['longitude'], img['latitude']
            image_df.at[index, 'geohash'] = pgh.encode(lat, lon)

In [5]:
# traindf = pd.read_csv('/raum_raid/rahul.agrawal/datasets/Jupiter_v4_3_updated_hdr_stops_corn_cotton_2021/master_annotations_pure_humananddustplusvt_mixeddisp_labelmapfix.csv')
traindf = pd.read_csv('/raum_raid/rakhil.immidisetti/datasets/Jupiter_vt_labelled_v1/master_annotations.csv')

In [5]:
get_geohash(traindf)


NameError: name 'get_geohash' is not defined

In [7]:
test_safety_v8 = pd.read_csv('/raum_raid/li.yu/data/critical_safety_dirty_2022_working_stereo_annotated_v8/master_annotations.csv')
test_caution_v13 = pd.read_csv('/raum_raid/li.yu/data/Jupiter_caution_dirty_v1.3/master_annotations.csv')
# test_productivity_v12 = pd.read_csv('/raum_raid/li.yu/data/Jupiter_productivity_dirty_v1.2/master_annotations.csv')
test_productivity_nimz_night = pd.read_csv('/raum_raid/rahul.agrawal/datasets/Jupiter_productivity_1018_nimz_night/master_annotations_nightdisp.csv')
test_productivity_nimz_day = pd.read_csv('/raum_raid/rahul.agrawal/datasets/Jupiter_productivity_1018_nimz_day/master_annotations_daydisp.csv')

get_geohash(test_safety_v8)
get_geohash(test_caution_v13)
# get_geohash(test_productivity_v12)
get_geohash(test_productivity_nimz_night)
get_geohash(test_productivity_nimz_day)

print(len(test_safety_v8))
print(len(test_caution_v13))
# print(len(test_productivity_v12))
print(len(test_productivity_nimz_day))
print(len(test_productivity_nimz_night))

/home/bluerivertech/rahul.agrawal/code/JupiterCVML/europa/base/src/europa/env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (30,143) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


26617
748
35307
46593


In [8]:
traindf['dataset'] = 'train'

test_safety_v8['dataset'] = 'test'
test_caution_v13['dataset'] = 'test'
# test_productivity_v12['dataset'] = 'test'
test_productivity_nimz_day['dataset'] = 'test'
test_productivity_nimz_night['dataset'] = 'test'

In [9]:
cols = ['geohash','operating_field_name','grower','id','dataset']

complete_left = pd.concat([traindf[cols],
                           test_safety_v8[cols],
                           test_caution_v13[cols],
                           test_productivity_nimz_day[cols],
                           test_productivity_nimz_night[cols]],
                           ignore_index=True)

In [10]:
complete_left.shape
complete_left['geohash_7'] = complete_left.geohash.str[:7]

In [11]:
complete_left.groupby(['geohash_7']).agg({'geohash':'count'})

,geohash
geohash_7,
9v7r8hc,40
9v7r8hf,19
9v7r8j1,71
9v7r8j3,3
9v7r8j5,43
9vb2335,29
9vb233h,4
9vb2367,42
9vb236k,12


In [12]:
# result = complete_left.groupby('geohash_7')['dataset'].sum().apply(lambda x: list(set(x))).reset_index()
dataset_groups = complete_left.groupby('geohash_7')['dataset'].sum().apply(lambda x: len(set(x))).reset_index()

dataset_groups.loc[dataset_groups['dataset'] == 5, 'hashtestgroup'] = 'train'
dataset_groups.loc[dataset_groups['dataset'] == 3, 'hashtestgroup'] = 'test'
dataset_groups.loc[dataset_groups['dataset'] == 7, 'hashtestgroup'] = 'train&test'

dataset_groups = dataset_groups.drop(['dataset'], axis=1)

print(dataset_groups.head())
print(len(dataset_groups))
print(dataset_groups.hashtestgroup.value_counts())


  geohash_7 hashtestgroup
0   9v7r8hc          test
1   9v7r8hf          test
2   9v7r8j1          test
3   9v7r8j3          test
4   9v7r8j5          test
375
test     311
train     64
Name: hashtestgroup, dtype: int64


In [13]:
# complete_left['geohash_7'] = complete_left.geohash.str[:7]
hash_gp = complete_left.groupby('geohash_7').agg({'geohash':'count'})
hash_gp.rename(columns={"geohash": "geohash_count"}, inplace = True)
hash_gp['percentage'] = (hash_gp['geohash_count'] / sum(hash_gp['geohash_count']))*100
hash_gp.reset_index(level=0, inplace=True)
hash_gp['bounds'] =  [geohash.bounds(geo) for geo in hash_gp['geohash_7']]
hash_gp['polygon_geom'] = [Polygon(zip([bounds.sw.lon, bounds.ne.lon, bounds.ne.lon, bounds.sw.lon],
                                       [bounds.ne.lat, bounds.ne.lat, bounds.sw.lat, bounds.sw.lat])) 
                           for bounds in hash_gp['bounds']]  
hash_gp.head()

,geohash_7,geohash_count,percentage,bounds,polygon_geom
0,9v7r8hc,40,0.035206,"((30.875701904296875, -96.67831420898438), (30...","POLYGON ((-96.67831420898438 30.8770751953125,..."
1,9v7r8hf,19,0.016723,"((30.875701904296875, -96.67694091796875), (30...","POLYGON ((-96.67694091796875 30.8770751953125,..."
2,9v7r8j1,71,0.062491,"((30.8770751953125, -96.67831420898438), (30.8...",POLYGON ((-96.67831420898438 30.87844848632812...
3,9v7r8j3,3,0.002640,"((30.878448486328125, -96.67831420898438), (30...",POLYGON ((-96.67831420898438 30.87982177734375...
4,9v7r8j5,43,0.037846,"((30.8770751953125, -96.67556762695312), (30.8...",POLYGON ((-96.67556762695312 30.87844848632812...


In [14]:
left_formatted_drop = complete_left.sort_values(by=['operating_field_name']).drop_duplicates(subset=['geohash_7'], keep='first')
hash_gp2 = hash_gp.merge(left_formatted_drop[['geohash_7','operating_field_name','grower','dataset']], how = 'left', on = 'geohash_7')

In [15]:
print(hash_gp2[hash_gp2['geohash_count'] > 90].shape[0])
print(hash_gp2[hash_gp2['percentage'] > 1.00].shape[0])
print(hash_gp2.geohash_7.nunique())

99
29
375


In [16]:
hash_gp2.columns

Index(['geohash_7', 'geohash_count', 'percentage', 'bounds', 'polygon_geom',
       'operating_field_name', 'grower', 'dataset'],
      dtype='object')

### Create geopandas df 

In [17]:
crs = {'init': 'epsg:4326'}
polygon_df = gpd.GeoDataFrame(hash_gp2, crs=crs, geometry= hash_gp2['polygon_geom'])
polygon_df.head()

/home/bluerivertech/rahul.agrawal/code/JupiterCVML/europa/base/src/europa/env/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geohash_7,geohash_count,percentage,bounds,polygon_geom,operating_field_name,grower,dataset,geometry
0,9v7r8hc,40,0.035206,"((30.875701904296875, -96.67831420898438), (30...","POLYGON ((-96.67831420898438 30.8770751953125,...",Goodland East 2D,David Codell,test,"POLYGON ((-96.67831 30.87708, -96.67694 30.877..."
1,9v7r8hf,19,0.016723,"((30.875701904296875, -96.67694091796875), (30...","POLYGON ((-96.67694091796875 30.8770751953125,...",Goodland East 2D,David Codell,test,"POLYGON ((-96.67694 30.87708, -96.67557 30.877..."
2,9v7r8j1,71,0.062491,"((30.8770751953125, -96.67831420898438), (30.8...",POLYGON ((-96.67831420898438 30.87844848632812...,Goodland East 2B,David Codell,test,"POLYGON ((-96.67831 30.87845, -96.67694 30.878..."
3,9v7r8j3,3,0.002640,"((30.878448486328125, -96.67831420898438), (30...",POLYGON ((-96.67831420898438 30.87982177734375...,Goodland East 2B,David Codell,test,"POLYGON ((-96.67831 30.87982, -96.67694 30.879..."
4,9v7r8j5,43,0.037846,"((30.8770751953125, -96.67556762695312), (30.8...",POLYGON ((-96.67556762695312 30.87844848632812...,Goodland East 2B,David Codell,test,"POLYGON ((-96.67557 30.87845, -96.67419 30.878..."


In [18]:
polygon_df2 = polygon_df.merge(dataset_groups, how = 'left', on = 'geohash_7')
polygon_df2.head()


,geohash_7,geohash_count,percentage,bounds,polygon_geom,operating_field_name,grower,dataset,geometry,hashtestgroup
0,9v7r8hc,40,0.035206,"((30.875701904296875, -96.67831420898438), (30...","POLYGON ((-96.67831420898438 30.8770751953125,...",Goodland East 2D,David Codell,test,"POLYGON ((-96.67831 30.87708, -96.67694 30.877...",test
1,9v7r8hf,19,0.016723,"((30.875701904296875, -96.67694091796875), (30...","POLYGON ((-96.67694091796875 30.8770751953125,...",Goodland East 2D,David Codell,test,"POLYGON ((-96.67694 30.87708, -96.67557 30.877...",test
2,9v7r8j1,71,0.062491,"((30.8770751953125, -96.67831420898438), (30.8...",POLYGON ((-96.67831420898438 30.87844848632812...,Goodland East 2B,David Codell,test,"POLYGON ((-96.67831 30.87845, -96.67694 30.878...",test
3,9v7r8j3,3,0.002640,"((30.878448486328125, -96.67831420898438), (30...",POLYGON ((-96.67831420898438 30.87982177734375...,Goodland East 2B,David Codell,test,"POLYGON ((-96.67831 30.87982, -96.67694 30.879...",test
4,9v7r8j5,43,0.037846,"((30.8770751953125, -96.67556762695312), (30.8...",POLYGON ((-96.67556762695312 30.87844848632812...,Goodland East 2B,David Codell,test,"POLYGON ((-96.67557 30.87845, -96.67419 30.878...",test


In [19]:
print(len(polygon_df2))
print(polygon_df2.hashtestgroup.value_counts())

375
test     311
train     64
Name: hashtestgroup, dtype: int64


In [20]:
polygon_df2.shape

(375, 10)

In [21]:
polygon_df2['geohash_7'].nunique()

375

In [22]:
polygon_df2['hashtestgroup'].value_counts()

test     311
train     64
Name: hashtestgroup, dtype: int64

In [23]:
#typical 100 acres midwest
area_per_geohash = (152.9 * 152.4)
total_area_covered = polygon_df2.geohash_7.nunique() * area_per_geohash
print(round(total_area_covered/1000,2), 'sq km')

8738.24 sq km


### Plot geohash polygons

In [24]:
poly_test = polygon_df2[polygon_df2['hashtestgroup'] == 'test']
poly_train = polygon_df2[polygon_df2['hashtestgroup'] == 'train']
poly_traintest = polygon_df2[polygon_df2['hashtestgroup'] == 'train&test']

In [25]:
import folium
m = folium.Map([32.60742, -91.45020], zoom_start=5)
for _, r in poly_test.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data= geo_j,
                           style_function=lambda x: {'fillColor': 'pink' if r['dataset'] == 'train' else 'blue',
                                                    'color': 'pink' if r['dataset'] == 'train' else 'blue',
                                                    })
    percent = round(r['percentage'],2)
    folium.Popup('geohash_7: {}, percentage {}'.format(r['geohash_7'],percent)).add_to(geo_j)
    folium.Tooltip('Grower: {}, operating_field_name: {}, count {}'.format(r['grower'],r['operating_field_name'], r['geohash_count'])).add_to(geo_j)
    geo_j.add_to(m)
                   
                 
# folium.LatLngPopup().add_to(m)
# folium.features.GeoJsonPopup(fields=row['geohash_count'],labels=False ).add_to(m)

folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite',
        overlay=False,
        control=True
    ).add_to(m)
m

In [26]:
import folium
m = folium.Map([32.60742, -91.45020], zoom_start=5)

for _, r in poly_test.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data= geo_j,
                           style_function=lambda x: {'fillColor': 'blue',
                                                    'color': 'blue',
                                                    })
    percent = round(r['percentage'],2)
    folium.Popup('geohash_7: {}, percentage {}'.format(r['geohash_7'],percent)).add_to(geo_j)
    folium.Tooltip('Grower: {}, operating_field_name: {}, count {}'.format(r['grower'],r['operating_field_name'], r['geohash_count'])).add_to(geo_j)
    geo_j.add_to(m)
    
for _, r in poly_train.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data= geo_j,
                           style_function=lambda x: {'fillColor': 'red' ,
                                                    'color': 'red',
                                                    })
    percent = round(r['percentage'],2)
    folium.Popup('geohash_7: {}, percentage {}'.format(r['geohash_7'],percent)).add_to(geo_j)
    folium.Tooltip('Grower: {}, operating_field_name: {}, count {}'.format(r['grower'],r['operating_field_name'], r['geohash_count'])).add_to(geo_j)
    geo_j.add_to(m)

for _, r in poly_traintest.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data= geo_j,
                           style_function=lambda x: {'fillColor': 'cyan' ,
                                                    'color': 'cyan',
                                                    })
    percent = round(r['percentage'],2)
    folium.Popup('geohash_7: {}, percentage {}'.format(r['geohash_7'],percent)).add_to(geo_j)
    folium.Tooltip('Grower: {}, operating_field_name: {}, count {}'.format(r['grower'],r['operating_field_name'], r['geohash_count'])).add_to(geo_j)
    geo_j.add_to(m)



folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite',
        overlay=False,
        control=True
    ).add_to(m)
m

In [27]:
# m.save("/home/bluerivertech/rahul.agrawal/test_train_geohash_v44vthumandust.html")

In [28]:
print(poly_train.geohash_count.sum())
print(poly_test.geohash_count.sum())
print(poly_traintest.geohash_count.sum())


4352
109265
0


In [29]:
t1 = '9vb82ms'
tl = '9vb82me'
tb = '9vb82mk'
tr = '9vb82mt'
tr2 = '9vb82mw'
from pygeodesy import geohash as gh
print(gh.distance_(t1, tr2))
print(gh.equirectangular_(t1, tr2))
# print(pgh.d (train_hash3, test_hash))


486.71
305.40640796254723


In [30]:
traindf['geohash_7'] = traindf.geohash.str[:7]
test_safety_v8['geohash_7'] = test_safety_v8.geohash.str[:7]
test_caution_v13['geohash_7'] = test_caution_v13.geohash.str[:7]
test_productivity_nimz_day['geohash_7'] = test_productivity_nimz_day.geohash.str[:7]
test_productivity_nimz_night['geohash_7'] = test_productivity_nimz_night.geohash.str[:7]



In [31]:
from pygeodesy import geohash as gh

def min_test_geohash_distance(target, hashlist):
    distances = [gh.equirectangular_(target, testhash) for testhash in hashlist]
    mindistance = min(distances)
    return mindistance

In [32]:
train_hashes = poly_train.geohash_7
test_hashes = poly_test.geohash_7
test_hashlist = [h for _, h in test_hashes.items()]

train_hash_distances = {}
traindf['min_test_geohash_distance'] = 0.0

ctr = 0
for _, train_hash in train_hashes.items():
    ctr += 1
    if ctr % 100 == 0:
        print(ctr)
    min_distance = min_test_geohash_distance(train_hash, test_hashlist)
    traindf.loc[traindf.geohash_7 == train_hash, 'min_test_geohash_distance'] = min_distance


In [33]:
traindf.min_test_geohash_distance.value_counts()

44118.542893    1033
44048.985487     451
43982.764298     259
44185.088329     174
44394.369622     155
45550.114107     150
44671.103232     143
44324.192902     115
44535.962088     111
44871.362944     107
43917.505180     107
44801.412904      90
44806.357197      80
45212.785897      76
44526.536599      74
44052.955592      71
44400.934797      70
44188.518511      69
45754.935761      68
45348.481078      66
44731.874771      61
44321.299343      55
43988.330713      53
44937.312477      53
45007.051302      48
45484.284100      44
44600.316290      41
44735.784273      35
44266.022400      34
45687.104166      33
45686.848970      33
44194.586641      33
45482.233388      32
44390.430051      31
44059.571640      30
44254.431815      30
45345.395763      29
45481.977043      21
45073.319643      19
45345.652880      18
45618.404979      16
45550.881989      15
44662.750468      15
45620.194001      14
45278.753134      12
45414.764663       8
44464.960006       6
44595.872036 

In [34]:
traindf.loc[100, 'min_test_geohash_distance']

44535.96208832609

In [35]:
distances = sorted(traindf.min_test_geohash_distance.value_counts().to_dict().keys())
print(len(distances))
print(distances[:10])
# len(trainv44vt[trainv44vt.min_test_geohash_distance > ])

64
[43847.09705400427, 43912.99182079028, 43917.50517979589, 43982.764297732705, 43988.33071291226, 44048.98548657446, 44052.95559237813, 44059.57164041308, 44118.54289313694, 44123.5637059744]


In [36]:
ok = traindf[traindf.min_test_geohash_distance > 300.0]
print(len(ok))
print(len(traindf))

4352
4352


In [ ]:
print(traindf.camera_location.value_counts())

In [ ]:
traindf.to_csv('/home/bluerivertech/rahul.agrawal/Jupiter_vt_labelled_v1_2021_geohashdistances_alltestsets.csv')

In [ ]:
traindf.special_notes.value_counts()